# Permissions Filter for KernelFunctions

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24229.4"
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging.Debug, 8.0.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.13.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.13.0-alpha"
#r "nuget: Microsoft.Extensions.Http, 8.0.0"
#r "nuget: Microsoft.Extensions.Http.Resilience"
#r "nuget: Microsoft.Extensions.Compliance.Redaction"

Installed Packages Microsoft.DotNet.Interactive, 1.0.0-beta.24229.4 Microsoft.Extensions.Compliance.Redaction, 8.5.0 Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.Extensions.Http, 8.0.0 Microsoft.Extensions.Http.Resilience, 8.5.0 Microsoft.Extensions.Logging, 8.0.0 Microsoft.Extensions.Logging.Console, 8.0.0 Microsoft.Extensions.Logging.Debug, 8.0.0 Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Plugins.OpenApi, 1.13.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.13.0-alpha

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel.Plugins.OpenApi;
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;
using Microsoft.Extensions.Compliance.Redaction;

## Add Function with KernelFunction

In [3]:
class Demographics
{
    [KernelFunction]
    public int GetPersonAge(string name)
    {
        return name switch
        {
            "Marco" => 30,
            _ => 20
        };
    }
}

In [4]:
var builder = Kernel.CreateBuilder();
var settings = new OpenAIPromptExecutionSettings() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };

## Add Permissions Filter for Functions

In [7]:
#pragma warning disable SKEXP0001
class PermissionFilter : IFunctionInvocationFilter
{
    public async Task OnFunctionInvocationAsync(FunctionInvocationContext context, Func<FunctionInvocationContext, Task> next)
    {
        Console.WriteLine($"Allow {context.Function.Name}?");
        if (await Microsoft.DotNet.Interactive.Kernel.GetInputAsync() == "y")
        {
            await next(context);
        }
        else
        {
            throw new Exception("Permission denied");
        }
        
    }
}


## Add to DI

In [8]:
#pragma warning disable SKEXP0001
builder.Services.AddSingleton<IFunctionInvocationFilter, PermissionFilter>();

In [9]:
Kernel kernel = builder
    .AddOpenAIChatCompletion("gpt-4o", Environment.GetEnvironmentVariable("OAI_KEY")!)
    .Build();
kernel.ImportPluginFromType<Demographics>();
var settings = new OpenAIPromptExecutionSettings() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
var chatService = kernel.GetRequiredService<IChatCompletionService>();
ChatHistory chat = new();

## Use Kernel Function and ask for Marco Age

In [10]:
while (true)
{
    Console.WriteLine("Q: ");
    var message = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Your prompt");
    chat.AddUserMessage(message);
    Console.WriteLine($"User: {message}");
    
    var response = await chatService.GetChatMessageContentAsync(chat, settings, kernel);

    Console.WriteLine(response);
    chat.Add(response);
}

Q: 
User: How old is Marco?
Allow GetPersonAge?
Marco is 30 years old.
Q: 


Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 72
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 46
   at Submission#10.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)